# Messaging Bot

## Download database from S3, update the data, and upload database to S3

1. Create user agent on Reddit API 
2. Extract hot posts and top posts creator IDs from Reddit API
3. Combine the two creator ID lists and remove the duplicates
4. Check the creator IDs if they already exists in the current creator ID database. If not, add them to the shuffle list
5. Shuffle the list to make sure every creator have the same probability to be chosen
6. Divide the list into three groups
7. Send different messages to three groups
8. Save new creator IDs in our creator ID database
9. upload the database to AWS S3

Todo:
shuffle list and send msgs to creators

In [1]:
import boto3
s3 = boto3.client('s3')
# s3.create_bucket(Bucket='top-hot-posts-creator-ids',
#     CreateBucketConfiguration={'LocationConstraint': 'us-west-1'})

In [2]:
s3.download_file('top-hot-posts-creator-ids','author_id.pkl', 'author_id.pkl')

In [3]:
import praw
import pandas as pd
from datetime import datetime
from praw.models import MoreComments

# Enter the message title and body
title1 = 'Hello'
title2 = 'Hi'
title3 = 'How are you?'

body1 = ''
body2 = ''
body3 = ''

# Enter your client id, secret and user agent
reddit = praw.Reddit(client_id = 'k_irONQxgcqWFg', 
                client_secret = 'A6udBWn-8PXi2p7X34K7HT5THiA', 
                user_agent = 'Test',
                username='AddyZhang',
                password='Haldis199444')

In [4]:
# Reddit groups that Drew wants us to look at
Reddit_groups = ['depression', 'anxiety', 'OCD', 'socialanxiety', 'panicdisorder']

# Create a dataframe to store text data
def hot_posts_to_df():
    # Write a loop to put hot post from all groups and their key info into one dataframe
    posts = []
    for group in Reddit_groups:
        mental_subreddit = reddit.subreddit(group)
        for post in mental_subreddit.hot(limit=10): # you can change the top number of posts 

            posts.append([group, post.title, post.url, post.id, post.author])

    posts = pd.DataFrame(posts, columns = ['group','title', 'url','post_id','author']) 
    
    return posts

In [5]:
def top_posts_to_df():
    # Write a loop to put hot post from all groups and their key info into one dataframe
    posts = []
    for group in Reddit_groups:
        mental_subreddit = reddit.subreddit(group)
        for post in mental_subreddit.top('day', limit=10): # you can change the top number of posts 

            posts.append([group, post.title, post.url, post.id, post.author])

    posts = pd.DataFrame(posts, columns = ['group','title', 'url','post_id','author']) 
    
    return posts

In [6]:
def combine_list(l1,l2):
    l3 = l1+l2
    return list(set(l3))

In [7]:
import pickle

def load_pickle(file_name):  
    
    with open(f'{file_name}.pkl', 'rb') as f:
        author_id_db = pickle.load(f)
    
    return author_id_db

In [8]:
def save_as_pickle(file_name, list_to_pickle):
    
    with open(f'{file_name}.pkl', 'wb') as f:
        pickle.dump(list_to_pickle, f)

In [9]:
def message(user_id, title, body):
    """
    Send private messages to redditor with the title and body
    """
    reddit.redditor(f'{user_id}').message(f'{title}', f'{body}')

In [10]:
import random

def shuffle_list(list_unsf):
    """
    Shuffle list to make sure everyone that has the same probabilty to be selected
    """
    random.shuffle(list_unsf)
    return list_unsf

In [11]:
def divide_group(list_sf):
    k = int(len(list_sf)/3) # divide into three groups
    group1 = list_sf[:k]
    group2 = list_sf[k:2*k]
    group3 = list_sf[2*k:]
    return group1, group2, group3

In [12]:
if __name__ == '__main__':
    
    hot_posts = hot_posts_to_df()
    hot_posts.author = hot_posts.author.astype('str') # change dtypes
    hot_author_id_list = hot_posts.author.tolist()
    
    top_posts = top_posts_to_df()
    top_posts.author = top_posts.author.astype('str') # change dtypes
    top_author_id_list = top_posts.author.tolist()
    
    author_id_list = combine_list(hot_author_id_list, top_author_id_list) # combine and remove duplicates
    file_name = 'author_id'
    author_id_db = load_pickle(file_name)
    
#     author_id_rs = [] # need this list for random sampling
    for author_id in author_id_list:
        if author_id not in author_id_db:
            
#             author_id_rs.append(author_id) # append the new author_id in our random sampling list
            author_id_db.append(author_id) # append the new author_id to our database list 
    
    
#     author_id_sf = shuffle_list(author_id_rs) # shuffle the creator id list

#     group1, group2, group3 = divide_group(author_id_sf) # divide into three groups
    
    # send msgs to three group users
#     for user_id in group1:
#         message(user_id,title1,body1)
    
#     for user_id in group2:
#         message(user_id,title2,body2)
    
#     for user_id in group3:
#         message(user_id,title3,body3)

#     print(group1,group2,group3)
    save_as_pickle(file_name, author_id_db)
    s3.upload_file('author_id.pkl','top-hot-posts-creator-ids', 'author_id.pkl')
    

In [13]:
print(author_id_db)

['xpanda16', 'Mental_senstrias', 'pvdilla123', 'madsadchadglad', 'Canucks_ZinccxFan', 'xSly_Foxx', 'circinia', 'Mumbawobz', 'tetrisphere69', 'Gerbilll', 'r36ei', 'patientlips', 'Bodhicahya', 'magentatroops', 'Wysa_ai', 'ilovemalakwaleed', 'moonstone28', 'codbla69', 't0niidk', 'ihateworkingandlife', 'invaderxanny', 'p0Oferdogmam', 'Vehicle_Voltron', 'SolomonNightrider', 'clappingenballs', 'szman135', 'Nervous-Tale', 'alyssaoftheeast', 'ImetalheadBGI', 'KikilooRose', 'GiltVoices', 'RedSoxCeltics', 'Yacinekoukou', 'socialanimal_us', 'suicidalsadgirl', 'MaterialSquid', 'princess-kelly', 'originalmoon', 'TeodrosKing', 'angelshum10', 'tinylilcaterpillar', 'blazdncnfsd', 'DrJanaScrivani', 'FatherSteph', 'Movingforwardtimes', 'OhSheGlows', 'twentysixants', 'MuchRedditLessTime', 'transparentfears', 'grozdeto', 'p3ras1', 'elabanana_', 'wilmygirl22', 'pikachuuuuu', 'Raven-OA_', 'SidTheNerd', 'throwaway83426', 'Weaselur', 'cezzatron', 'jacobamcdonald', 'haha_ur_mom_gay_haha', 'loveandsqualor4', 'l